In [1]:
import sys

sys.path.append("..")

In [2]:
import copy

import numpy as np
from gensim.models import FastText
from gensim.models.fasttext import save_facebook_model

from quantitizer.utils import mb
from quantitizer.integration.gensim.fasttext import make_new_fasttext_model


ft = FastText.load_fasttext_format("../data/cc.en.300.bin").wv
vocab = list(ft.key_to_index.keys())
mb(ft)

ModuleNotFoundError: No module named 'cuml'

In [17]:
import quantitizer as q
import importlib

importlib.reload(q)

def get_quantitized(ft, sub_size):
    ft_quantitized = make_new_fasttext_model(
        ft, q.quantitize(ft.vectors, sub_size=sub_size, n_iter=5),
        q.quantitize(ft.vectors_ngrams, sub_size=sub_size, n_iter=5))
    return ft_quantitized

In [3]:
sub_sizes = [2, 50, 100]
quantitized_fts = []
for sub_size in sub_sizes:
    quantitized_fts.append(get_quantitized(ft, sub_size))
    print(f"Sub size = {sub_size} -> memory = {mb(quantitized_fts[-1])}")
    quantitized_fts[-1].save(f"../out/compressed_{sub_size}")

642.8218088150024

In [18]:
ft_quantitized = get_quantitized(ft, 2)
# ft_quantitized.save("com")
mb(ft_quantitized)

In [13]:
import importlib
importlib.reload(temp)

from gensim.models.fasttext import FastTextKeyedVectors

# ft_quantitized = temp.CompressedFastTextKeyedVectors.load("out/compressed")
ft_quantitized = temp.FastTextKeyedVectors.load("out/compressed")
# ft_quantitized = FastText.load("out/compressed")

In [1]:
from quantitizer.integration.evaluation import vecs_similarity

q = vecs_similarity(ft, ft_quantitized, vocab)
q

ModuleNotFoundError: No module named 'quantitizer'

In [24]:
from scipy.spatial import distance

distance.cosine(original_vec, new_ft.get_vector("sz"))

0.42782288789749146

In [8]:
# import pickle

# with open("out/original.bin", 'wb') as file:
#     pickle.dump(ft, file)

# ft.save("out/original")

# save_facebook_model(ft, "out/original.bin")

In [10]:
FastText.load("out/original")

IndexError: index 500000 is out of bounds for axis 0 with size 500000

In [75]:
new_ft = copy.deepcopy(ft)

In [76]:
new_ft.wv.vectors_ngrams.shape

(500000, 300)

In [77]:
from quantitizer import quantitize

new_ft.wv.vectors_ngrams = quantitize(new_ft.wv.vectors_ngrams, sub_size=2)
new_ft.wv.vectors = quantitize(new_ft.wv.vectors, sub_size=2)

KeyboardInterrupt: 

In [59]:
compressed_vec = new_ft.wv.get_vector("sz")

In [60]:
from scipy.spatial import distance

distance.cosine(original_vec, compressed_vec)

0

In [69]:
with open("out/compressed.bin", 'wb') as file:
    pickle.dump(new_ft, file)
# save_facebook_model(new_ft, "out/compressed.bin")

In [74]:
ft.wv.vectors.nbytes // 1024 // 1024

2288

In [73]:
ft.wv.vectors_ngrams.nbytes // 1024 // 1024

572

In [72]:
new_ft.wv.vectors_ngrams.nbytes // 1024 // 1024

4